In [ ]:
from __future__ import annotations

import datetime
from time import sleep
from typing import Any

from diskcache import Cache
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import tool
from pydantic import BaseModel, Field
from tavily import TavilyClient

from chain_reaction.caching import cache_calls
from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.utils import get_structured_response

# Load API keys
api_keys = APIKeys()

# Define temporary disk cache for caching tool calls
cache = Cache()

# Simple example with mock tool

In [ ]:
@tool
@cache_calls(cache=cache)
def tool_with_cache(x: float) -> float:
    """A tool that squares a number with simulated delay."""
    sleep(2)  # Simulate a time-consuming computation
    return x * x

In [ ]:
tool_with_cache.get_name()

In [ ]:
# Invoke the tool for first time (will take ~2 seconds)
tool_with_cache.invoke({"x": 3.0})

In [ ]:
# Invoke the tool to see caching in action (will be instantaneous)
tool_with_cache.invoke({"x": 3.0})

In [ ]:
# Get the value directly from the cache
cache.get(key="2a613a05fab2d2cc4e56aea0e7e26871", tag=True)

# Cached web searches

In [ ]:
# Initialize Tavily client
tavily_client = TavilyClient(api_key=api_keys.tavily.get_secret_value())

In [ ]:
# Define a tool for searching the web using Tavily with caching
@tool
@cache_calls(cache=cache)
def search_web(query: str) -> dict[str, Any]:
    """Performs a web search using Tavily.

    Args:
        query (str): The search query.

    Returns:
        dict[str, Any]: The search results.
    """
    return tavily_client.search(query=query)


search_web.invoke({"query": "When will it snow next in Centennial, CO?"})

In [ ]:
# Search again, and get from cache
search_web.invoke({"query": "When will it snow next in Centennial, CO?"})

# Web search agent with caching

In [ ]:
# Initialize a chat model
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.factual().model_dump(),
)


# Create a response model
class WeatherResult(BaseModel):
    """Response model for weather results."""

    forecast_date: datetime.date | None = Field(description="The date of the weather forecast.")
    chance_of_snow: float | None = Field(description="The chance of snow in the specified location.", ge=0, le=100)
    amount_of_snow: float | None = Field(description="The expected amount of snow in inches.", ge=0)
    temperature: float | None = Field(description="The expected temperature in Fahrenheit.")
    snow_start: int | None = Field(
        description="The hour when snow is expected to start (24-hour format). None if no snow is expected.",
        ge=0,
        le=23,
    )
    snow_end: int | None = Field(
        description="The hour when snow is expected to end (24-hour format). None if no snow is expected.", ge=0, le=23
    )

    @classmethod
    def get_result(cls, response: dict[str, Any]) -> WeatherResult:
        """Parse the weather result from the model response."""
        result = get_structured_response(model=cls, response=response)
        if result is None:
            raise ValueError("Failed to parse weather result from response.")
        return result


# Initialize an agent using the chat model & tools
agent = create_agent(
    model=init_chat_model(
        model=ModelName.CLAUDE_HAIKU,
        timeout=None,
        max_retries=2,
        api_key=api_keys.anthropic,
        **ModelBehavior.deterministic().model_dump(),
    ),
    tools=[search_web],
    system_prompt="""
    You're a helpful assistant that can search the web for weather information.
    Use the provided web search tools to answer user questions accurately.
    ONLY use the tools when necessary to find up-to-date weather information (2-3 searches max per conversation).
    If you can't find information on a specific field, populate it with None.
    But try to find as much information as possible.
    """,
    response_format=WeatherResult,
)

In [ ]:
# Invoke the agent to get weather information
response = agent.invoke(input={"messages": [HumanMessage(content="When will it snow next in Centennial, CO?")]})
WeatherResult.get_result(response=response)

In [ ]:
# Invoke again to see if the agent uses any of the same tool calls
response = agent.invoke(input={"messages": [HumanMessage(content="When will it snow next in Centennial, CO?")]})
WeatherResult.get_result(response=response)